In [1]:
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
import win32com.client
from SmartApi import SmartConnect
from logzero import logger
import requests
import pandas as pd
import os

In [2]:
api_key = 'FYBFTPOl'
username = 'AAAF620196'
pwd = '0054'

smartApi = SmartConnect(api_key)
try:
    token = "YS4MOR7OUZEHGNV3VFOQ5TVMRE" 
    totp = pyotp.TOTP(token).now()
except Exception as e:
    logger.error("Invalid Token: The provided token is not valid.")
    raise e

correlation_id = "abcde"
data = smartApi.generateSession(username, pwd, totp)

if data['status'] == False:
    logger.error(data)
    
else:
    # login api call
    # logger.info(f"You Credentials: {data}")
    authToken = data['data']['jwtToken']
    refreshToken = data['data']['refreshToken']
    # fetch the feedtoken
    feedToken = smartApi.getfeedToken()
    # fetch User Profile
    res = smartApi.getProfile(refreshToken)
    smartApi.generateToken(refreshToken)
    res=res['data']['exchanges']

[I 250603 09:45:04 smartConnect:124] in pool


[E 250603 09:45:04 smartConnect:221] Error occurred while making a POST request to https://apiconnect.angelone.in/rest/auth/angelbroking/user/v1/loginByPassword. Headers: {'Content-type': 'application/json', 'X-ClientLocalIP': '127.0.0.1', 'X-ClientPublicIP': '106.193.147.98', 'X-MACAddress': 'eb:f2:cf:fa:da:c0', 'Accept': 'application/json', 'X-PrivateKey': 'FYBFTPOl', 'X-UserType': 'USER', 'X-SourceID': 'WEB'}, Request: {'clientcode': 'AAAF620196', 'password': '0054', 'totp': '292025'}, Response: HTTPSConnectionPool(host='apiconnect.angelone.in', port=443): Max retries exceeded with url: /rest/auth/angelbroking/user/v1/loginByPassword (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))


SSLError: HTTPSConnectionPool(host='apiconnect.angelone.in', port=443): Max retries exceeded with url: /rest/auth/angelbroking/user/v1/loginByPassword (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))

In [ ]:
symbol_token_map = {}
bank_nifty_stocks = []
file_path = r"C:\\Users\\jaiad\\Pro T\\indices_dashboard.xlsx"

In [ ]:
# Fetch Angel One's scrip master data
url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df = pd.DataFrame(data)
df.columns = df.columns.str.lower()  # Ensure all lowercase for consistency

# Display available columns (optional)
print("Available Columns:", df.columns.tolist())

# Helper function to display filtered tokens
def show_token_info(label, filt_df):
    print(f"\n{label}:")
    if not filt_df.empty:
        print(filt_df[['symbol', 'token', 'exch_seg']].drop_duplicates())
    else:
        print("❌ Not Found")

# Nifty Spot
nifty_spot = df[(df['name'].str.upper() == 'NIFTY') & (df['exch_seg'] == 'NSE')]
show_token_info("Nifty Spot", nifty_spot)

# Nifty Futures
nifty_fut = df[(df['name'].str.upper() == 'NIFTY') & (df['exch_seg'] == 'NFO') & (df['symbol'].str.contains("FUT"))]
show_token_info("Nifty Futures", nifty_fut)

# Nifty Bank Spot
banknifty_spot = df[(df['name'].str.upper() == 'BANKNIFTY') & (df['exch_seg'] == 'NSE')]
show_token_info("Nifty Bank Spot", banknifty_spot)

# Nifty Bank Futures
banknifty_fut = df[(df['name'].str.upper() == 'BANKNIFTY') & (df['exch_seg'] == 'NFO') & (df['symbol'].str.contains("FUT"))]
show_token_info("Nifty Bank Futures", banknifty_fut)

# Gift Nifty (look for 'GIFT' or 'SGXNIFTY' keywords)
gift_nifty = df[df['name'].str.upper().str.contains('GIFT') | df['symbol'].str.upper().str.contains('GIFT')]
show_token_info("Gift Nifty", gift_nifty)

# India VIX
vix = df[df['name'].str.upper().str.contains("VIX")]
show_token_info("India VIX", vix)

Available Columns: ['token', 'symbol', 'name', 'expiry', 'strike', 'lotsize', 'instrumenttype', 'exch_seg', 'tick_size']

Nifty Spot:
        symbol     token exch_seg
38    Nifty 50  99926000      NSE
6052     NIFTY     26000      NSE

Nifty Futures:
                symbol  token exch_seg
40324  NIFTY31JUL25FUT  53216      NFO
42179  NIFTY26JUN25FUT  56785      NFO
45070  NIFTY28AUG25FUT  64103      NFO

Nifty Bank Spot:
          symbol     token exch_seg
37    Nifty Bank  99926009      NSE
2752   BANKNIFTY     26009      NSE

Nifty Bank Futures:
                    symbol  token exch_seg
40863  BANKNIFTY31JUL25FUT  53213      NFO
42177  BANKNIFTY26JUN25FUT  56782      NFO
42684  BANKNIFTY28AUG25FUT  64100      NFO

Gift Nifty:
❌ Not Found

India VIX:
       symbol     token exch_seg
44  India VIX  99926017      NSE


In [ ]:
# ---------- FETCH CANDLE DATA ----------
def fetch_candle_data(symbol, token, exchange, interval):
    now = datetime.now()
    market_open = now.replace(hour=9, minute=15, second=0, microsecond=0)
    market_close = now.replace(hour=15, minute=30, second=0, microsecond=0)

    def get_last_trading_day(current_date):
            last_trading_day = current_date - timedelta(days=1)
            # If today is Monday, the last trading day is Friday
            if current_date.weekday() == 0:  # Monday
                last_trading_day = current_date - timedelta(days=3)
            # If today is Sunday, the last trading day is Friday
            elif current_date.weekday() == 6:  # Sunday
                last_trading_day = current_date - timedelta(days=2)
            # If today is Saturday, the last trading day is Friday
            elif current_date.weekday() == 5:  # Saturday
                last_trading_day = current_date - timedelta(days=1)
            return last_trading_day
    
    # Check if today is Saturday (weekday == 5) or Sunday (weekday == 6)
    if now.weekday() >= 5:
        print("⚠️ Market is closed on Saturday and Sunday.")
        # Fetch last traded OHLC data (from the last trading day, typically Friday)
        last_trading_day = get_last_trading_day(now)
        from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
        to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
    else:
        if now < market_open:
            print("⚠️ Market has not opened yet. Try again after 9:15 AM.")
            # Fetch last traded OHLC data (from the previous trading day)
            last_trading_day = get_last_trading_day(now)
            from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
            to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
        elif now > market_close:
            print("⚠️ Market is closed for the day.")
            # Fetch last traded OHLC data (from today's market close)
            from_date = (market_close - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
            to_date = market_close.strftime("%Y-%m-%d %H:%M")
        else:
            # Market is open, fetch live data
            from_date = (now - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
            to_date = now.strftime("%Y-%m-%d %H:%M")

    params = {
        "exchange": exchange,
        "symboltoken": token,
        "interval": interval,
        "fromdate": from_date,
        "todate": to_date
    }

    try:
        response = smartApi.getCandleData(params)
        if response['status'] and response['data']:
            data = response['data']
            # Return only the last/latest candle
            latest = data[-1:]
            df = pd.DataFrame(latest, columns=["Datetime", "Open", "High", "Low", "Close", "Volume"])
            df["Symbol"] = symbol
            df["Interval"] = interval
            return df
        else:
            print(f"⚠️ No candle data for {symbol} ({interval})")
            return pd.DataFrame()
    except Exception as e:
        print(f"❌ Error fetching {symbol} [{interval}]: {e}")
        return pd.DataFrame()


In [ ]:
# ---------- TIME INTERVALS ----------
time_intervals = [
    "ONE_MINUTE", "THREE_MINUTE", "FIVE_MINUTE",
    "FIFTEEN_MINUTE", "THIRTY_MINUTE", "ONE_HOUR"
]

# ---------- TOKENS FROM IMAGE ----------
instruments = {
    "NIFTY": {"token": "99926000", "exchange": "NSE"},
    "NIFTY FUTURE": {"token": "56785", "exchange": "NFO"},
    "BANKNIFTY": {"token": "99926009", "exchange": "NSE"},
    "BANKNIFTY FUTURE": {"token": "56782", "exchange": "NFO"},
    "INDIA VIX": {"token": "99926017", "exchange": "NSE"}  # Based on standard VIX token
}
# ---------- COLLECT AND DISPLAY ----------
final_df = pd.DataFrame()
for symbol, info in instruments.items():
    for interval in time_intervals:
        df = fetch_candle_data(symbol, info["token"], info["exchange"], interval)
        final_df = pd.concat([final_df, df], ignore_index=True)

# Optional: sort and show
final_df = final_df[["Symbol", "Interval", "Datetime", "Open", "High", "Low", "Close", "Volume"]]
final_df.sort_values(by=["Symbol", "Interval", "Datetime"], inplace=True)

# ---------- DISPLAY ----------
pd.set_option('display.max_rows', None)
print(final_df.sort_index())

⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market has not opened yet. Try again after 9:15 AM.
⚠️ Market 

In [ ]:
# Step 2: Map intervals to simplified labels
interval_mapping = {
    "ONE_MINUTE": "1min",
    "THREE_MINUTE": "3min",
    "FIVE_MINUTE": "5min",
    "FIFTEEN_MINUTE": "15min",
    "THIRTY_MINUTE": "30min",
    "ONE_HOUR": "1hr"
}
df = final_df.sort_index()

def calculate_sentiment(row):
    open_price = row["Open"]
    close_price = row["Close"]
    symbol = row["Symbol"]
    interval = interval_mapping[row["Interval"]]

    # General sentiment calculation
    change_percent = ((close_price - open_price) / open_price) * 100
    if change_percent > 0.1:
        return "bullish"
    elif change_percent < -0.1:
        return "bearish"
    else:
        return "buying/selling/neutral"

df["Sentiment"] = df.apply(calculate_sentiment, axis=1)
print(df)

              Symbol        Interval                   Datetime      Open  \
0              NIFTY      ONE_MINUTE  2025-05-29T15:29:00+05:30  24882.70   
1              NIFTY    THREE_MINUTE  2025-05-29T15:27:00+05:30  24870.60   
2              NIFTY     FIVE_MINUTE  2025-05-29T15:25:00+05:30  24855.80   
3              NIFTY  FIFTEEN_MINUTE  2025-05-29T15:15:00+05:30  24850.45   
4              NIFTY   THIRTY_MINUTE  2025-05-29T15:15:00+05:30  24850.45   
5              NIFTY        ONE_HOUR  2025-05-29T15:15:00+05:30  24850.45   
6       NIFTY FUTURE      ONE_MINUTE  2025-05-29T15:29:00+05:30  24834.10   
7       NIFTY FUTURE    THREE_MINUTE  2025-05-29T15:27:00+05:30  24832.90   
8       NIFTY FUTURE     FIVE_MINUTE  2025-05-29T15:25:00+05:30  24835.00   
9       NIFTY FUTURE  FIFTEEN_MINUTE  2025-05-29T15:15:00+05:30  24831.40   
10      NIFTY FUTURE   THIRTY_MINUTE  2025-05-29T15:15:00+05:30  24831.40   
11      NIFTY FUTURE        ONE_HOUR  2025-05-29T15:15:00+05:30  24831.40   

In [ ]:
excel_filename = "indices_sentiment.xlsx"
excel_path = os.path.abspath(excel_filename)

excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = True  # Show Excel in real-time

if os.path.exists(excel_path):
    wb = excel.Workbooks.Open(excel_path)
    print(f"Opened existing workbook: {excel_path}")
else:
    wb = excel.Workbooks.Add()
    wb.SaveAs(excel_path)
    print(f"Created new workbook: {excel_path}")

ws = wb.Sheets(1)  # Always use the first sheet

# Step 5: Set up the structure to match the screenshot
ws.Range("A1").Value = "Date: 27-05-2025"
ws.Range("A2").Value = "Time: 15:15"

# Column headers (Row 4)
headers = ["", "", "1min", "3min", "5min", "15min", "30min", "1hrs"]
for col_idx, header in enumerate(headers, 1):
    ws.Cells(4, col_idx).Value = header

# Row labels (Column A)
row_labels = {6: "A", 8: "B", 10: "C", 12: "D", 14: "F"}
indice_names = { 6: "BANKNIFTY", 8: "BANKNIFTY FUTURE", 10: "NIFTY", 12: "GIFT NIFTY", 14: "VIX" }

for col in range(2, 9):  # Columns B to H (2 to 8)
    ws.Columns(col).ColumnWidth = 20  # Adjust width (increase if needed)
    ws.Columns(col).WrapText = True

for row, label in row_labels.items():
    ws.Cells(row, 1).Value = label
for row, label in indice_names.items():
    ws.Cells(row, 2).Value = label

# Step 6: Apply row coloring
# Light gray for data rows (BANKNIFTY, BANKNIFTY FUTURE, NIFTY, GIFT NIFTY, VIX)
# Base row pairs for gradient: current row (gray) and the one below it
gradient_pairs = [6, 8, 10, 12, 14]
base_color = 0xD3D3D3

# Function to darken color slightly
def darken_color(hex_color, step):
    r = (hex_color & 0xFF0000) >> 16
    g = (hex_color & 0x00FF00) >> 8
    b = (hex_color & 0x0000FF)
    
    # Decrease brightness (keep minimum at 0)
    r = max(0, r - step)
    g = max(0, g - step)
    b = max(0, b - step)
    
    # Recombine into hex
    return (r << 16) + (g << 8) + b

# Apply gradient coloring
current_color = base_color
for top_row in gradient_pairs:
    for col in range(1, 11):  # Columns A to J
        # Top row: light gray
        ws.Cells(top_row, col).Interior.Color = current_color
        ws.Cells(top_row+1, col).Interior.Color = current_color
        # Bottom row: slightly darker
        darker_color = darken_color(current_color, 15)
    current_color = darker_color  # Move to next gradient level
    
# Step 7: Map data to specific cells and apply text coloring
cell_mapping = {
    "BANKNIFTY": {"1min": "C6", "3min": "D6", "5min": "E6", "15min": "F6", "30min": "G6", "1hr": "H6"},
    "BANKNIFTY FUTURE": {"1min": "C8", "3min": "D8", "5min": "E8", "15min": "F8", "30min": "G8", "1hr": "H8"},
    "NIFTY": {"1min": "C10", "3min": "D10", "5min": "E10", "15min": "F10", "30min": "G10", "1hr": "H10"},
    "INDIA VIX": {"1min": "C14", "3min": "D14", "5min": "E14", "15min": "F14", "30min": "G14", "1hr": "H14"}
}

# Step 8: Fill in the sentiment values with text coloring
for symbol in cell_mapping.keys():
    for interval, cell in cell_mapping[symbol].items():
        # Find the sentiment for this symbol and interval
        interval_key = [k for k, v in interval_mapping.items() if v == interval][0]
        sentiment = df[(df["Symbol"] == symbol) & (df["Interval"] == interval_key)]["Sentiment"].iloc[0]
        
        # Write to the cell
        print(f"Filling {cell} with {sentiment} for {symbol} ({interval})...")
        ws.Range(cell).Value = sentiment
        
        # Apply text coloring based on sentiment
        if "bullish" in sentiment:
            ws.Range(cell).Font.Color = 0x008000  # Green for bullish
        elif "bearish" in sentiment:
            ws.Range(cell).Font.Color = 0x0000FF  # Red for bearish
        else:
            ws.Range(cell).Font.Color = 0x000000  # Black for buying/selling/neutral

# Step 9: Save the workbook
wb.Save()  # Save changes to the already opened file
print("Excel file 'market_sentiment.xlsx' has been created and filled successfully.")

Opened existing workbook: C:\Users\jaiad\Pro T\Final_draft1\indices_sentiment.xlsx
Filling C6 with buying/selling/neutral for BANKNIFTY (1min)...
Filling D6 with buying/selling/neutral for BANKNIFTY (3min)...
Filling E6 with bullish for BANKNIFTY (5min)...
Filling F6 with bullish for BANKNIFTY (15min)...
Filling G6 with bullish for BANKNIFTY (30min)...
Filling H6 with bullish for BANKNIFTY (1hr)...
Filling C8 with buying/selling/neutral for BANKNIFTY FUTURE (1min)...
Filling D8 with buying/selling/neutral for BANKNIFTY FUTURE (3min)...
Filling E8 with buying/selling/neutral for BANKNIFTY FUTURE (5min)...
Filling F8 with buying/selling/neutral for BANKNIFTY FUTURE (15min)...
Filling G8 with buying/selling/neutral for BANKNIFTY FUTURE (30min)...
Filling H8 with buying/selling/neutral for BANKNIFTY FUTURE (1hr)...
Filling C10 with buying/selling/neutral for NIFTY (1min)...
Filling D10 with buying/selling/neutral for NIFTY (3min)...
Filling E10 with bullish for NIFTY (5min)...
Filling F10 

In [2]:
# Initialize Excel workbook once
def initialize_excel():
    excel_filename = "indices_sentiment.xlsx"
    excel_path = os.path.abspath(excel_filename)
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = True  # Show Excel in real-time

    if os.path.exists(excel_path):
        wb = excel.Workbooks.Open(excel_path)
        print(f"Opened existing workbook: {excel_path}")
    else:
        wb = excel.Workbooks.Add()
        wb.SaveAs(excel_path)
        print(f"Created new workbook: {excel_path}")

    ws = wb.Sheets(1)  # Use first sheet
    setup_excel_structure(ws)
    return excel, wb, ws

# Set up Excel structure (headers, labels, and formatting)
def setup_excel_structure(ws):
    # Set date and time (will be updated in main loop)
    ws.Range("A1").Value = f"Date: {datetime.now().strftime('%d-%m-%Y')}"
    ws.Range("A2").Value = f"Time: {datetime.now().strftime('%H:%M')}"

    # Column headers (Row 4)
    headers = ["", "", "1min", "3min", "5min", "15min", "30min", "1hrs"]
    for col_idx, header in enumerate(headers, 1):
        ws.Cells(4, col_idx).Value = header

    # Row labels and indice names
    row_labels = {6: "A", 8: "B", 10: "C", 12: "D", 14: "F"}
    indice_names = {6: "BANKNIFTY", 8: "BANKNIFTY FUTURE", 10: "NIFTY", 12: "GIFT NIFTY", 14: "VIX"}

    for col in range(2, 9):  # Columns B to H
        ws.Columns(col).ColumnWidth = 20
        ws.Columns(col).WrapText = True

    for row, label in row_labels.items():
        ws.Cells(row, 1).Value = label
    for row, label in indice_names.items():
        ws.Cells(row, 2).Value = label

    # Apply gradient coloring
    gradient_pairs = [6, 8, 10, 12, 14]
    base_color = 0xD3D3D3

    def darken_color(hex_color, step):
        r = (hex_color & 0xFF0000) >> 16
        g = (hex_color & 0x00FF00) >> 8
        b = (hex_color & 0x0000FF)
        r = max(0, r - step)
        g = max(0, g - step)
        b = max(0, b - step)
        return (r << 16) + (g << 8) + b

    current_color = base_color
    for top_row in gradient_pairs:
        for col in range(1, 11):  # Columns A to J
            ws.Cells(top_row, col).Interior.Color = current_color
            ws.Cells(top_row + 1, col).Interior.Color = current_color
        current_color = darken_color(current_color, 15)

# Fetch and process candle data
def fetch_and_process_data():
    final_df = pd.DataFrame()
    for symbol, info in instruments.items():
        for interval in time_intervals:
            df = fetch_candle_data(symbol, info["token"], info["exchange"], interval)
            final_df = pd.concat([final_df, df], ignore_index=True)

    final_df = final_df[["Symbol", "Interval", "Datetime", "Open", "High", "Low", "Close", "Volume"]]
    final_df.sort_values(by=["Symbol", "Interval", "Datetime"], inplace=True)
    return final_df

# Calculate sentiment
def calculate_sentiment(df):
    interval_mapping = {
        "ONE_MINUTE": "1min",
        "THREE_MINUTE": "3min",
        "FIVE_MINUTE": "5min",
        "FIFTEEN_MINUTE": "15min",
        "THIRTY_MINUTE": "30min",
        "ONE_HOUR": "1hr"
    }

    def sentiment_logic(row):
        open_price = row["Open"]
        close_price = row["Close"]
        change_percent = ((close_price - open_price) / open_price) * 100
        if change_percent > 0.1:
            return "bullish"
        elif change_percent < -0.1:
            return "bearish"
        else:
            return "buying/selling/neutral"

    df["Sentiment"] = df.apply(sentiment_logic, axis=1)
    df["Interval"] = df["Interval"].map(interval_mapping)
    return df

def update_excel(ws, df):
    cell_mapping = {
        "BANKNIFTY": {"1min": "C6", "3min": "D6", "5min": "E6", "15min": "F6", "30min": "G6", "1hr": "H6"},
        "BANKNIFTY FUTURE": {"1min": "C8", "3min": "D8", "5min": "E8", "15min": "F8", "30min": "G8", "1hr": "H8"},
        "NIFTY": {"1min": "C10", "3min": "D10", "5min": "E10", "15min": "F10", "30min": "G10", "1hr": "H10"},
        "INDIA VIX": {"1min": "C14", "3min": "D14", "5min": "E14", "15min": "F14", "30min": "G14", "1hr": "H14"}
    }

    # Update date and time
    ws.Range("A1").Value = f"Date: {datetime.now().strftime('%d-%m-%Y')}"
    ws.Range("A2").Value = f"Time: {datetime.now().strftime('%H:%M')}"

    # Fill in sentiment values with text coloring
    for symbol in cell_mapping.keys():
        for interval, cell in cell_mapping[symbol].items():
            # Find the sentiment for this symbol and interval
            try:
                sentiment = df[(df["Symbol"] == symbol) & (df["Interval"] == interval)]["Sentiment"].iloc[0]
            except IndexError:
                print(f"Warning: No sentiment data for {symbol} ({interval})")
                continue

            # Write to the cell
            print(f"Filling {cell} with {sentiment} for {symbol} ({interval})...")
            ws.Range(cell).Value = sentiment

            # Apply text coloring based on sentiment
            if "bullish" in sentiment.lower():
                ws.Range(cell).Font.Color = 0x008000  # Green for bullish
            elif "bearish" in sentiment.lower():
                ws.Range(cell).Font.Color = 0x0000FF  # Blue for bearish (original used blue, not red)
            else:
                ws.Range(cell).Font.Color = 0x000000  # Black for buying/selling/neutral

    # Save the workbook
    wb = ws.Parent  # Get the workbook from the worksheet
    wb.Save()
    print("Excel file updated successfully.")

In [4]:
# Main loop to run the functions every 60 seconds
def main():
    try:
        # Initialize Excel once
        excel, wb, ws = initialize_excel()

        while True:
            try:
                # Fetch and process data
                df = fetch_and_process_data()
                # Calculate sentiment
                df = calculate_sentiment(df)
                # Update Excel
                update_excel(ws, df)
                print(f"Data updated at {datetime.now().strftime('%H:%M:%S')}")
            except Exception as e:
                print(f"Error during data update: {str(e)}")
                if isinstance(e, requests.exceptions.ConnectionError):
                    print("Connection error detected. Retrying in 60 seconds...")
            # Wait for 60 seconds
            time.sleep(60)

    except KeyboardInterrupt:
        print("Script interrupted by user.")
        # Clean up Excel application
        try:
            wb.Save()
            wb.Close()
            excel.Quit()
            print("Excel application closed successfully.")
        except Exception as e:
            print(f"Error closing Excel: {str(e)}")
        sys.exit(0)
    except Exception as e:
        print(f"Fatal error: {str(e)}")
        # Clean up Excel application
        try:
            wb.Close()
            excel.Quit()
        except:
            pass
        sys.exit(1)

Final Encapsulation : 

In [8]:
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
import win32com.client
from SmartApi import SmartConnect
from logzero import logger
import requests
import os

class IndicesDashboard:
    def __init__(self, api_key, username, pwd, token, file_name="indices_sentiment.xlsx"):
        self.api_key = api_key
        self.username = username
        self.pwd = pwd
        self.token = token
        self.smartApi = SmartConnect(self.api_key)
        self.authenticate()
        self.file_name = file_name
        self.file_path = os.path.abspath(self.file_name)
        self.time_intervals = [
            "ONE_MINUTE", "THREE_MINUTE", "FIVE_MINUTE",
            "FIFTEEN_MINUTE", "THIRTY_MINUTE", "ONE_HOUR"
        ]
        self.instruments = {
            "BANKNIFTY": {"token": "99926009", "exchange": "NSE"},
            "BANKNIFTY FUTURE": {"token": "56782", "exchange": "NFO"},
            "NIFTY": {"token": "99926000", "exchange": "NSE"},
            "INDIA VIX": {"token": "99926017", "exchange": "NSE"}
        }
        self.interval_mapping = {
            "ONE_MINUTE": "1min",
            "THREE_MINUTE": "3min",
            "FIVE_MINUTE": "5min",
            "FIFTEEN_MINUTE": "15min",
            "THIRTY_MINUTE": "30min",
            "ONE_HOUR": "1hr"
        }
        self.excel, self.wb, self.ws = self.initialize_excel()

    def authenticate(self):
        try:
            totp = pyotp.TOTP(self.token).now()
        except Exception as e:
            logger.error("Invalid Token: The provided token is not valid.")
            raise e
        data = self.smartApi.generateSession(self.username, self.pwd, totp)
        if data['status'] == False:
            logger.error(data)
            raise Exception("Authentication failed")
        else:
            self.authToken = data['data']['jwtToken']
            self.refreshToken = data['data']['refreshToken']
            self.feedToken = self.smartApi.getfeedToken()
            self.profile = self.smartApi.getProfile(self.refreshToken)
            self.smartApi.generateToken(self.refreshToken)

    def initialize_excel(self):
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = True
        if os.path.exists(self.file_path):
            wb = excel.Workbooks.Open(self.file_path)
            print(f"Opened existing workbook: {self.file_path}")
        else:
            wb = excel.Workbooks.Add()
            wb.SaveAs(self.file_path)
            print(f"Created new workbook: {self.file_path}")
        ws = wb.Sheets(1)
        self.setup_excel_structure(ws)
        return excel, wb, ws

    def setup_excel_structure(self, ws):
        ws.Range("A1").Value = f"Date: {datetime.now().strftime('%d-%m-%Y')}"
        ws.Range("A2").Value = f"Time: {datetime.now().strftime('%H:%M')}"
        headers = ["", "", "1min", "3min", "5min", "15min", "30min", "1hrs"]
        for col_idx, header in enumerate(headers, 1):
            ws.Cells(4, col_idx).Value = header
        row_labels = {6: "A", 8: "B", 10: "C", 12: "D", 14: "F"}
        indice_names = {6: "BANKNIFTY", 8: "BANKNIFTY FUTURE", 10: "NIFTY", 12: "GIFT NIFTY", 14: "VIX"}
        for col in range(2, 9):
            ws.Columns(col).ColumnWidth = 20
            ws.Columns(col).WrapText = True
        for row, label in row_labels.items():
            ws.Cells(row, 1).Value = label
        for row, label in indice_names.items():
            ws.Cells(row, 2).Value = label
        gradient_pairs = [6, 8, 10, 12, 14]
        base_color = 0xD3D3D3
        def darken_color(hex_color, step):
            r = (hex_color & 0xFF0000) >> 16
            g = (hex_color & 0x00FF00) >> 8
            b = (hex_color & 0x0000FF)
            r = max(0, r - step)
            g = max(0, g - step)
            b = max(0, b - step)
            return (r << 16) + (g << 8) + b
        current_color = base_color
        for top_row in gradient_pairs:
            for col in range(1, 11):
                ws.Cells(top_row, col).Interior.Color = current_color
                ws.Cells(top_row + 1, col).Interior.Color = current_color
            current_color = darken_color(current_color, 15)

    def fetch_candle_data(self, symbol, token, exchange, interval):
        now = datetime.now()
        market_open = now.replace(hour=9, minute=15, second=0, microsecond=0)
        market_close = now.replace(hour=15, minute=30, second=0, microsecond=0)
        def get_last_trading_day(current_date):
            last_trading_day = current_date - timedelta(days=1)
            if current_date.weekday() == 0:
                last_trading_day = current_date - timedelta(days=3)
            elif current_date.weekday() == 6:
                last_trading_day = current_date - timedelta(days=2)
            elif current_date.weekday() == 5:
                last_trading_day = current_date - timedelta(days=1)
            return last_trading_day
        if now.weekday() >= 5:
            last_trading_day = get_last_trading_day(now)
            from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
            to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
        else:
            if now < market_open:
                last_trading_day = get_last_trading_day(now)
                from_date = last_trading_day.replace(hour=14, minute=30).strftime("%Y-%m-%d %H:%M")
                to_date = last_trading_day.replace(hour=15, minute=30).strftime("%Y-%m-%d %H:%M")
            elif now > market_close:
                from_date = (market_close - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = market_close.strftime("%Y-%m-%d %H:%M")
            else:
                from_date = (now - timedelta(hours=1)).strftime("%Y-%m-%d %H:%M")
                to_date = now.strftime("%Y-%m-%d %H:%M")
        params = {
            "exchange": exchange,
            "symboltoken": token,
            "interval": interval,
            "fromdate": from_date,
            "todate": to_date
        }
        try:
            response = self.smartApi.getCandleData(params)
            if response['status'] and response['data']:
                data = response['data']
                latest = data[-1:]
                df = pd.DataFrame(latest, columns=["Datetime", "Open", "High", "Low", "Close", "Volume"])
                df["Symbol"] = symbol
                df["Interval"] = interval
                return df
            else:
                print(f"⚠️ No candle data for {symbol} ({interval})")
                return pd.DataFrame()
        except Exception as e:
            print(f"❌ Error fetching {symbol} [{interval}]: {e}")
            return pd.DataFrame()

    def fetch_and_process_data(self):
        final_df = pd.DataFrame()
        for symbol, info in self.instruments.items():
            for interval in self.time_intervals:
                df = self.fetch_candle_data(symbol, info["token"], info["exchange"], interval)
                final_df = pd.concat([final_df, df], ignore_index=True)
        final_df = final_df[["Symbol", "Interval", "Datetime", "Open", "High", "Low", "Close", "Volume"]]
        final_df.sort_values(by=["Symbol", "Interval", "Datetime"], inplace=True)
        return final_df

    def calculate_sentiment(self, df):
        def sentiment_logic(row):
            open_price = row["Open"]
            close_price = row["Close"]
            change_percent = ((close_price - open_price) / open_price) * 100
            if change_percent > 0.1:
                return "bullish"
            elif change_percent < -0.1:
                return "bearish"
            else:
                return "buying/selling/neutral"
        df["Sentiment"] = df.apply(sentiment_logic, axis=1)
        df["Interval"] = df["Interval"].map(self.interval_mapping)
        return df

    def update_excel(self, ws, df):
        cell_mapping = {
            "BANKNIFTY": {"1min": "C6", "3min": "D6", "5min": "E6", "15min": "F6", "30min": "G6", "1hr": "H6"},
            "BANKNIFTY FUTURE": {"1min": "C8", "3min": "D8", "5min": "E8", "15min": "F8", "30min": "G8", "1hr": "H8"},
            "NIFTY": {"1min": "C10", "3min": "D10", "5min": "E10", "15min": "F10", "30min": "G10", "1hr": "H10"},
            "INDIA VIX": {"1min": "C14", "3min": "D14", "5min": "E14", "15min": "F14", "30min": "G14", "1hr": "H14"}
        }
        ws.Range("A1").Value = f"Date: {datetime.now().strftime('%d-%m-%Y')}"
        ws.Range("A2").Value = f"Time: {datetime.now().strftime('%H:%M')}"
        for symbol in cell_mapping.keys():
            for interval, cell in cell_mapping[symbol].items():
                try:
                    sentiment = df[(df["Symbol"] == symbol) & (df["Interval"] == interval)]["Sentiment"].iloc[0]
                except IndexError:
                    print(f"Warning: No sentiment data for {symbol} ({interval})")
                    continue
                ws.Range(cell).Value = sentiment
                if "bullish" in sentiment.lower():
                    ws.Range(cell).Font.Color = 0x008000  # Green
                elif "bearish" in sentiment.lower():
                    ws.Range(cell).Font.Color = 0x0000FF  # Blue
                else:
                    ws.Range(cell).Font.Color = 0x000000  # Black
        wb = ws.Parent
        wb.Save()
        print("Excel file updated successfully.")

    def run(self):
        try:
            while True:
                try:
                    df = self.fetch_and_process_data()
                    df = self.calculate_sentiment(df)
                    self.update_excel(self.ws, df)
                    print(f"Data updated at {datetime.now().strftime('%H:%M:%S')}")
                except Exception as e:
                    print(f"Error during data update: {str(e)}")
                    if isinstance(e, requests.exceptions.ConnectionError):
                        print("Connection error detected. Retrying in 60 seconds...")
                time.sleep(60)
        except KeyboardInterrupt:
            print("Script interrupted by user.")
            try:
                self.wb.Save()
                self.wb.Close()
                self.excel.Quit()
                print("Excel application closed successfully.")
            except Exception as e:
                print(f"Error closing Excel: {str(e)}")
        except Exception as e:
            print(f"Fatal error: {str(e)}")
            try:
                self.wb.Close()
                self.excel.Quit()
            except:
                pass


In [ ]:
dashboard = IndicesDashboard(
    api_key='FYBFTPOl',
    username='AAAF620196',
    pwd='0054',
    token='YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    file_name="indices_sentiment.xlsx"
)
dashboard.run()

[I 250618 10:10:34 smartConnect:124] in pool


Opened existing workbook: c:\Users\jaiad\Pro T\Final_draft1\indices_sentiment.xlsx


[E 250618 10:10:57 smartConnect:246] Error occurred while making a POST request to https://apiconnect.angelone.in/rest/secure/angelbroking/historical/v1/getCandleData. Error: Something Went Wrong, Please Try After Sometime. URL: https://apiconnect.angelone.in/rest/secure/angelbroking/historical/v1/getCandleData, Headers: {'Content-type': 'application/json', 'X-ClientLocalIP': '127.0.0.1', 'X-ClientPublicIP': '106.193.147.98', 'X-MACAddress': 'd9:33:98:14:6d:c5', 'Accept': 'application/json', 'X-PrivateKey': 'FYBFTPOl', 'X-UserType': 'USER', 'X-SourceID': 'WEB'}, Request: {'exchange': 'NSE', 'symboltoken': '99926017', 'interval': 'ONE_HOUR', 'fromdate': '2025-06-18 09:10', 'todate': '2025-06-18 10:10'}, Response: {'message': 'Something Went Wrong, Please Try After Sometime', 'errorcode': 'AB1004', 'status': False, 'data': None}


⚠️ No candle data for INDIA VIX (ONE_HOUR)
Excel file updated successfully.
Data updated at 10:10:58
Excel file updated successfully.
Data updated at 10:12:21
Excel file updated successfully.
Data updated at 10:13:43
Excel file updated successfully.
Data updated at 10:14:59
Excel file updated successfully.
Data updated at 10:16:14
